# 챕터 16: 코루틴

* 코루틴으로 작동하는 제너레이터의 동작과 상태
* 데커레이터를 이용해서 코루틴을 자동으로 기동하기
* 제너레이터 객체의 close()와 throw() 메서드를 통해 호출자가 코루틴을 제어하는 방법
* 종료할 때 코루틴이 값을 반환하는 방법
* 새로운 yoeld from 구문의 사용법과 으미
* 사용 예: 시뮬레이션의 동시 활동을 관리하기 위한 코루틴

## 16.1 코루틴은 제너레이터에서 어떻게 진화했는가?

## 16.2 코루틴으로 사용되는 제너레이터의 기본 동작

In [ ]:
# 가장 간단한 코루틴 사용 예
def simple_coroutine():
    print('-> coroutine started')
    x = yield # 단지 호출자에서 데이터를 받도록 설계하면 yield는 값을 생성하지 않는다. yield 키워드 뒤에 아무런 표현식이 없을 때 ㄱ밧을 생성하지 않으려는 의도를 암묵적으로 표현한다.
    print('-> coroutine received:', x)
    
my_coro = simple_coroutine()
my_coro
next(my_coro) # 제너레이터의 send() 메서드를 호출해서 코루틴 본체 안의 yield 문의 값을 42로 만든다. 이제 코루틴이 실행을 재개해서 다름 yield 문이 나오거나 종료될 때까지 실행한다.
my_coro.send(42) # 여기서는 제어 흐름이 코루틴 본체의 끝에 도달하므로, 일반적인 제너레이터와 마찬가지로 StopIteration 예외를 발생시킨다.

* 'GEN_CREATED': 실행을 시작하기 위해 대기하고 있는 상태
* 'GEN_RUNNING': 현재 인터프리터가 실행하고 있는 상태
* 'GEN_SUSPENDED': 현재 yield 문에서 대기하고 있는 상태
* 'GEN_CLOSED': 실행이 완료된 상태

In [ ]:
def simple_coro2(a):
    print('-> Started: a =', a)
    b = yield a
    print('-> Received: b =', b)
    c = yield a + b
    print('-> Received: c =', c)
    
my_coro2 = simple_coro2(14)
from inspect import getgeneratorstate
getgeneratorstate(my_coro2)
next(my_coro2)
getgeneratorstate(my_coro2)
my_coro2.send(28)
my_coro2.send(99)
getgeneratorstate(my_coro2)

## 16.3 예제: 이동 평균을 계산하는 코루틴

In [ ]:
# coroaverager0.py: 이동 편균 코루틴 코드
def averager():
    total = 0.0
    count = 0
    average = None
    while True: # 이 yield 문은 코루틴을 중단하고, 지금까지의 평균을 생성하기 위해 사용된다. 나중에 호출자가 이 코루틴에 값을 보내면 루프를 다시 실행한다.
        term = yield average
        total += term
        count += 1
        average = total/count

## 16.4 코루틴을 기동하기 위한 데커레이터

In [ ]:
# coroutil.py: 코루틴을 기동하기 위한 데커레이터
from functools import wraps

def coroutine(func):
    """데커레이터: 'func'를 기동해서 첫 번째 'yield'까지 진핸한다."""
    @wraps(func)
    def primer(*args, **kwargs): # 데커레이트된 제너레이터 함수는 primer() 함수로 치환되며, 실행하면 기동된 제너레이터를 반환한다.
        gen = func(*args, **kwargs) # 데커레이트된 함수를 호출해서 제너레이터 객체를 가져온다.
        next(gen)
        return gen
    return primer

In [ ]:
coro_avg = averager() # 코루틴 객체를 생성한다.
next(coro_avg) # next()를 호출해서 코루틴을 기동시킨다.
coro_avg.sned(10)
coro_avg.send(30)
coro_avg.send(5)

"""
    from coroutill import coroutine
    
    @coroutine
    def averager():
        total = 0.0
        count = 0
        average = None
        while True:
            term = yield average
            total += term
            count += 1
            average = total/count
"""

## 16.5 코루틴 종료와 예외처리

In [ ]:
# 처리하지 않은 예외에 의한 코루틴 종료
from coroaverager1 import averager
coro_avg = averager()
coro_avg.send(40) # @coroutine으로 데커레이트된 averager()를 사용하므로, 코루틴에 바로 값을 보낼 수 있다.
coro_avg.send(50)
coro_avg.send('spam') #비수치형 값을 보내면 코루틴 안에서 예외가 발생한다.
coro_avg.send(60) # 코루틴 안에서 예외를 처리하지 않으므로 코루틴이 종료된다. 이후에 코루틴을 다시 활성화하려면 StopIteration 예외가 발생한다.

* generator.throw(exc_type[.exc_value[,traceback]]): 제너레이터가 중단한 곳의 yield 표현식에 예외를 전달한다. 제너레이터가 예외를 처리하면, 제어흐름이 다음 yield 문까지 진행하고, 생성된 값은 generator.throw() 호출 값이 된다. 제너레이터가 예외를 처리하지 않으면 호출자까지 예외가 전파된다.

* generator.close(): 제너레이터가 실행을 중단한 yield 표현식이 GeneratorExit 예외를 GeneratorExit 예외를 처리하지 않거나 StopIteration 예외(일반적으로 제너레이터가 실행을 완료할 때 발생한다)를 발생시키면, 아무런 에러도 호출자에 전달되지 않는다. GeneratorExit 예외를 받으면 제너레이터는 아무런 값도 생성하지 않아야 한다. 아니면 RuntimeError 예외가 발생한다. 제너레이터에서 발생하는 다른 예외는 모두 호출자에 전달된다.

In [ ]:
# coro_exc_demo.py: 코루틴의 예외 처리 방법을 설명하기 위한 제너레이터
class DemoException(Exception):
    """설명에 사용할 예외 유형"""
    
    def demo_exc_handling():
        print('-> coroutine started')
        while True:
            try:
                x = yield
            except DemoException: # DemoException 예외를 따로 처리한다.
                print('*** DemoException handled. Continuing...')
            else: # 예외가 발생하지 않으면 받은 값을 출력한다.
                print('-> coroutine received: {!r}'.format(x))
        raise RuntimeError('This line should never run.') # 이 코드는 결코 실행되지 않는다.

In [ ]:
# 예외를 발생시키지 않는 demo_exc_handling()의 활성화 및 종료
exc_coro = demo_exc_handling()
next(exc_coro)
exc_coro.send(11)
exc_coro.send(22)
exc_coro.close()
from inspect import getgeneratorstate
getgeneratorstate(exc_coro)

In [ ]:
# DemoException을 demo_exc_handling() 안에 던져도 종료되지 않는다.
exc_coro = demo_exc_handling()
next(exc_coro)
exc_coro.send(11)
exc_coro.throw(DemoException)
getgenerator(exc_coro)

In [ ]:
exc_coro= demo_exc_handling()
next(exc_coro)
exc_coro.send(11)
exc_coro.throw(ZeroDivisionError)
getgenerator(exc_coro)

In [ ]:
class DemoException(Exception):
    """설명에 사용할 예외 유형"""
    
    def demo_finally():
        print('-> coroutine started')
        try:
            while True:
                try:
                    x = yield
                except DemoException:
                    print('*** DemoException handled. Continuing...')
                else:
                    print('-> coroutine received: {!r}'.format(x))
            finally:
                print('-> coroutine ending')      

## 16.6 코루틴에서 값 반환하기

In [ ]:
# coroaverager2.py: 결과를 반환하는 averager()코루틴 코드
from collections import namedtuple

Result = namedtuple('Result', 'count average')

def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield
        if term is None:
            break # 값을 반환하려면 코루틴이 정상적으로 종료되어야 한다. 그렇기 때문에 이 averager 버전에서는 루프를 빠져나오는 조건을 검사한다.
        total += term
        count += 1
        average = total/count
    return Result(count, average) # count와 average를 가진 namedtuple을 반환한다.

In [ ]:
coro_avg = averager()
next(coro_avg)
coro_avg.send(10) # 이 버전은 값을 생성하지 않는다.
coro_avg.send(30)
coro_avg.send(6.5)
coro_avg(None) # None을 보내면 루프를 빠져나오고 코루틴이 결과를 반환하면서 종료하게 된다. 일반적인 제너레이터 객체와 마찬가지로 StopIteration 예외가 발생한다. 예외 객체의 value 속성에는 반환된 값이 들어 있다.

In [ ]:
# StopIteration을 잡으면 averager()가 반환한 값을 가져올 수 있다.
coro_avg = averager()
nect(coro_avg)
coro_avg.send(10)
coro_avg.send(30)
coro_avg.send(6.5)
try:
    coro_avg.send(None)
except StopIteration as exc:
    result = exc.value
    
result

## 16.7 yield from 사용하기

In [ ]:
def gen():
    for c in 'AB':
        yield c
    for i in range(1,3):
        yield i
list(gen()) # ['A', 'B', 1, 2]

In [ ]:
# 위 코드를 아래의 코드로 바꿀 수 있다.
def gen():
    yield from 'AB'
    yield from range(1,3)
    
list(gen())

In [ ]:
# yield from으로 반복형 객체를 연결하기
def chain(*iterables):
    for it in iterables:
        yield from it
        
s = ;ABC
t = tuple(range(3))
list(chain(s, t))

* 대표 제너레이터:yield from <반복형> 표현식을 담고 있는 제너레이터 함수
* 하위 제너레이터: yield from 표현식 중 <반복형>에서 가져오는 제너레이터. PEP 380의 제목 '하위 제너레이터에 위임하기 위한 구문'에서 말하는 제너레이터가 바로 이것.
* 호출자: PEP380은 대표 제너레이터를 호출하는 코드를 '호출자'라고 표현한다. 문맥에 따라서 필자는 대표 제너레이터와 구분하기 위해 '호출자' 대신 '클라이언트'라는 용어를 사용하기도 한다. 하위 제너레이터 입장에서 보면 대표 제너레이터도 호출자기 때문이다.

In [ ]:
# coroaverager3.py: yield from을 이용해서 averager()를 구동하고 보고서 생성하기
from collections import namedtuple

Result = namedtuple('Result', 'count average')

# 하위 제너레이터
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield
        if term is None:
            break # 값을 반환하려면 코루틴이 정상적으로 종료되어야 한다. 그렇기 때문에 이 averager 버전에서는 루프를 빠져나오는 조건을 검사한다.
        total += term
        count += 1
        average = total/count
    return Result(count, average)

# 대표 제너레이터
def grouper(results, key):
    while True:
        results[key] = yield from averager()
        
# 호출자
def main(data):
    results = {}
    for key, values in data.items():
        group = grouper(results, key)
        next(group)
        for value in values:
            group.send(value)
        group.send(None) # 이 부분 중요하다!
    
    #print(results) # 디버깅하려면 이 줄의 주석을 해제하라.
    report(results)
    
# 실행 결과 보고서
def report(results):
    for key, result in sorted(results.items()):
        group, unit = key.split(';')
        print('{:2} {:5} averaging {:.2f}{}'.format(result.count, group, result.average, unit))

data = {
    'girls;kg': [40.9, 38.5, 44.3, 42.2, 45.2, 41.7, 44.5, 38.0, 40.6, 44.5],
    'girls;m': [1.6, 1.51, 1.4, 1.3, 1.41, 1.39, 1.33, 1.46, 1.45, 1.43],
    'boys;kg': [39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3],
    'boys;m': [ 1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46]
}

if __name__ == '__main__':
    main(data)

## 16.8 yield from의 의미
**그 반복자가 또 다른 제너레이터인 경우, 하위 제너레이터의 본체가 yield from 표현식의 대상 안에 들어가는 것과 동일한 효과가 발생한다. 게다가 하위 제너레이터는 값을 가진 return문을 이용해서 값을 반환할 수 있고, 그 값은 yield from 표현식의 값이 된다.**

- 앞의 예제에서 알 수 있는 것
    * 하위 제너레이터가 생성하는 값은 모두 대표 제어레이터의 호출자(즉, 클라이언트)에 바로 전달된다.
    * send()를 통해 대표 제너레이터에 전달한 값은 모두 하위 제너레이터에 직접 전달된다. 값이 None이면 하위 제너레이터의 __next__() 메서드가 호출된다. 전달된 값이 None이 아니면 하위 제너레이터의 send() 메서드가 호출된다. 호출된 메서드에서 StopIteration 예외가 발생하면 대표 제너레이터의 실행이 재개된다. 그 외의 예외는 대표 제너레이터에 전달된다.
    * 제너레이터나 하위 제너레이터에서 return expr 문을 실행하면, 제너레이터를 빠져나온 후 StopIteration(expr) 예외가 발생한다.
    * 하위 제너레이터가 실행을 마친 후 발생한 StopIteration 예외의 첫 번째 인수가 yield from 표현식의 값이 된다.
    
- yield from의 나머지 특징 두 가지는 예외와 종료에 관련되어 있다.
    * 대표 제너레이터에 던져진 GeneratorExit 이외의 예외는 하위 제너레이터의 throw() 메서드에 전달된다. throw() 메서드를 호출해서 StopIteration 예외가 발생하면 대표 제너레이터의 실행이 재개된다. 그 외의 예외는 대표 제너레이터에 전달된다.
    * GeneratorExit 예외가 대표 제너레이터에 던져지거나 대표 제너레이터의 close() 메서드가 호출되면 하위 제너레이터의 close() 메서드가 호출된다. 그 결과 예외가 발생하면 발생한 예외가 대표 제너레이터에 전파된다. 그렇지 않으면 대표 제너레이터에서 GeneratorExit 예외가 발생한다.

In [ ]:
# 대표 제너레이터 안의 RESULT = yield fromEXPR 문에 해당하는 단순화한 의사코드, 여기서는 단순한 가정에 따라 thro()와 close() 메서드를 지원하지 않고, StopIteration 예외만 지원한다.

_i = iter(EXPR)
try:
    _y = next(_i) # 하위 제너레이터를 기동시킨다. 반환된 값은 _y에 저장되어 최초의 생성 값으로 사용된다.
except StopIteration as _e:
    _r = _e.value # StopIteration이 발생하면 예외 객체에서 value 속성을 꺼내 _r에 할당한다. 이 값이 가장 간단한 경우의 RESULT 값이 된다.
else:
    while 1:
        _s = yield _y # 하위 제너레이터에서 생성된 값을 그대로 생성하고, 호출자가 보낼 _s를 기다린다. 이 코드 안에서 유일하게 사용된 yield 문이라는 점에 주의하라.
        try:
            _y = _i.send(_s) # 호출자가 보낸 _s를 하위 제너레이터에 전달하면서 하위 제너레이터의 실행을 진행시킨다.
        except StopIteration as _e: # 하위 제너레이터가 StopIteration 예외를 발생시키면, 예외 객체 안의 value 속성을 가져와서 _r에 할당하고, 루프를 빠져나온 후. 대표 제너레이터의 실행을 재개한다.
            _r = _e.value
            break
            
RESULT = _r

* _i(iterator): 하위 제너레이터
* _y(yielded): 하위 제너레이터가 생성한 값
* _r(result): 최종 결과값(즉, 하위 제너레이터가 종료된 후 yield from 표현식의 값)
* _s(sent): 호출자가 대표 제너레이터에 보낸 ㄱ밧. 하위 제너레이터에 전달된다.
* _e(exception): 예외(이 간단한 의사코드에서는 StopIteration 객체만 발생한다.)

In [ ]:
# 대표 제너레이터에서 RESULTT = yiled from EXPR 문과 대등한 의사코드
_i = iter(EXPR)
try:
    _y = next(_i)
except StopIteration as _e:
    _r = _e.value
else:
    while 1:
        try:
            _s = yield _y
            except GeneratorExit as _e:
                try:
                    _m = _i.close
                except GeneratorExit as _e:
                    try:
                        _m = _i.close
                    except AttributeError:
                        pass
                    else:
                        _m()
                    raise _e
                except BaseException as _e:
                    _x = sys.exc_info()
                    try:
                        _m = _i.throw
                    except AttributeError:
                        raise _e
                    else:
                        try:
                            _y = _m(*_x)
                        except StopIteration as _e:
                            _r = _e.value
                            break
                else:
                    try:
                        if _s is None:
                            _y = next(_i)
                        else:
                            _y = _i.send(_s)
                    except StopIteration as _e:
                        _r = _e.value
                        break
RESULT = _r